We want the neural net to be given the same size volume at each episode.



First, we upsample the SWC files to make sure to get rid of huge inter-node distances, which can cause problems when trying to render a volume.

Before running the `resample_swc` script, this are the steps that Yang took:

He went to 'v3d_tools/release_plugins/v3d_plugins/resample_swc' and ran `qmake` and then `make`. He made some symbolic links to the main `v3d_external` folder.

Then, to run the `resample_swc` script, simply run `/path/to/v3d_external/bin/vaa3d -x resample_swc -f resample_swc -i <input_file> -o <output_file>`.

We need to get the resample_swc file directory

we check to see if the branch is at least as big as the target volume before we sample nodes from it.

I tried minimum dimension cut-off of 25, but no branches met that criteria.

In [1]:
from swc_io import get_fnames_and_abspath_from_dir, swc_to_dframe, resample_swc, dframe_to_swc
import numpy as np
import pandas as pd

BRANCHES_DIR = "../data/03_human_branches_splitted"
UPSAMPLED_BRANCHES_DIR = "../data/04_human_branches_filtered_upsampled"

In [ ]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

vaa3d_path = filedialog.askopenfilename(title='Select compiled Vaa3d binary')

fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
total_dim = 0
for i in range(len(fnames)):
    # upsample the SWC
    upsampled_fpath = resample_swc(fnames[i], abs_paths[i], vaa3d_path)
  

In [7]:
# tested size 25, but no branches met this criteria
TARGET_DIM_SIZE = 10.  # size in pixels for all x,y,z dims
half_size = TARGET_DIM_SIZE / 2
#min_swc_dim_span = TARGET_DIM_SIZE + 5  # some leeway to make things easy
padding = TARGET_DIM_SIZE//2

print("filtering branches that have any dimension smaller than ", TARGET_DIM_SIZE)

print("USING BRANCHES instead of filtered branches!")
fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
assert(len(fnames)>0)
total_dim = 0
total_skipped = 0
for i in range(len(fnames)):
    
    if (i % 1000 == 0):
        print("testing branch #", i, "avg span: ", total_dim/3)
    
    # replace nodes with our upsampled nodes
    nodes = swc_to_dframe(abs_paths[i])
    
    
    
        
    try:
        assert(isinstance(nodes, pd.DataFrame))
    except:
        total_skipped += 1
        continue
    
    x_span = nodes.x.max() - nodes.x.min()
    y_span = nodes.y.max() - nodes.y.min()
    z_span = nodes.z.max() - nodes.z.min()
    
    total_dim = total_dim + x_span + y_span + z_span
    
    dim_size_good = np.array([x_span, y_span, z_span]) > TARGET_DIM_SIZE
    #if z_span > TARGET_DIM_SIZE:   print("found z bigger than {}: {}".format(int(TARGET_DIM_SIZE), z_span))
    if np.all(dim_size_good):  # our SWC meets our criteria
        #print("{} is big enough".format(i))
        
        # upsample the SWC
        #upsampled_fpath = resample_swc(fnames, abs_paths[i])
        # replace nodes with our upsampled nodes
        #nodes = swc_to_dframe(upsampled_fpath)
        
        # find nodes far enough away from bounds
        central_nodes = nodes.loc[(nodes.x.between(nodes.x.min()+padding,nodes.x.max()+padding)) &
                              (nodes.y.between(nodes.y.min()+padding,nodes.y.max()+padding)) &
                              (nodes.z.between(nodes.z.min()+padding,nodes.z.max()+padding))]
        
        if len(central_nodes) > 0:
            # sample more than once from same swc file
            for j in range(50):
                # sample a central node
                center_node = central_nodes.sample(n=1)
                # grab all nodes in neighborhood of center_node
                sample = nodes.loc[(nodes.x.between(center_node.x.item()-half_size, center_node.x.item()+half_size)) &
                                  (nodes.y.between(center_node.y.item()-half_size, center_node.y.item()+half_size)) &
                                  (nodes.z.between(center_node.z.item()-half_size, center_node.z.item()+half_size))]

                # save sample to swc
                #print("saving ", fnames[i])
                dframe_to_swc(fnames[i]+"sample{}".format(j), sample)
    else:  # SWC does not have large enough dims
        #print("dims too small ", x_span, y_span, z_span) 
        pass
    
print("total skipped:", total_skipped)
    
    

filtering branches that have any dimension smaller than  10.0
USING BRANCHES instead of filtered branches!
testing branch # 0 avg span:  0.0


/home/shit/bin/anaconda3/lib/python3.6/site-packages/pandas/io/formats/format.py:1586: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  self.obj = self.obj.loc[:, cols]
/home/shit/bin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


testing branch # 1000 avg span:  34398.1253333
testing branch # 2000 avg span:  67684.9713333
testing branch # 3000 avg span:  100417.61
testing branch # 4000 avg span:  135599.097
testing branch # 5000 avg span:  168135.668
testing branch # 6000 avg span:  203019.533667
testing branch # 7000 avg span:  234844.360333
testing branch # 8000 avg span:  268357.728333
testing branch # 9000 avg span:  304192.899667
testing branch # 10000 avg span:  336945.003333
testing branch # 11000 avg span:  372532.318333
testing branch # 12000 avg span:  406559.440667
testing branch # 13000 avg span:  439785.407333
testing branch # 14000 avg span:  475205.946333
testing branch # 15000 avg span:  511078.612667
testing branch # 16000 avg span:  542001.683667
testing branch # 17000 avg span:  576585.393333
testing branch # 18000 avg span:  611765.019
testing branch # 19000 avg span:  646309.223
testing branch # 20000 avg span:  680056.589333
testing branch # 21000 avg span:  713208.363
testing branch # 220

Next, we want to only randomly sample from nodes sufficiently far away from the boundary

In [ ]:
# sample from the SWC
# first, we won't pick nodes too close to the edges
#filtered_nodes = np.where()
#print("dim sizes are good")

for swc_file in good_swc_paths[:1]: # TODO remove slice
    nodes = good_swcs[swc_file]
    good_nodes = 